In [3]:
!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import joblib

class DestinationRecommendationModel:
    def __init__(self):
        self.model = None
        self.df = None
        self.tfidf_matrix = None
        self.tfidf_vectorizer = None

    def preprocess_data(self, data):
        data['combined_features'] = data.apply(lambda x: ' '.join(x[['Traveler age', 'Traveler gender', 'Traveler nationality',
                                                                     'Accommodation type','Transportation type', 'Budget']].astype(str)), axis=1)
        return data

    def train_model(self, data):
        self.df = self.preprocess_data(data)
        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.df['combined_features'])
        self.model = cosine_similarity(self.tfidf_matrix, self.tfidf_matrix)

    def save_checkpoint(self, filename):
        joblib.dump((self.model, self.df, self.tfidf_matrix, self.tfidf_vectorizer), filename)

    def load_checkpoint(self, filename):
        self.model, self.df, self.tfidf_matrix, self.tfidf_vectorizer = joblib.load(filename)

    def recommend_destination(self, user_preferences, top_n=5):
        if self.model is None:
            raise Exception("Model has not been trained or loaded.")

        # Extract the user preferences
        user_features = ' '.join([str(value) for value in user_preferences.values()])
        user_tfidf = self.tfidf_vectorizer.transform([user_features])

        # Compute similarity between user preferences and destinations
        user_destination_similarity = cosine_similarity(user_tfidf, self.tfidf_matrix)

        sim_scores = user_destination_similarity[0]
        sim_scores = list(enumerate(sim_scores))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[:top_n]  # Consider top n destinations
        trip_indices = [i[0] for i in sim_scores]

        recommended_destinations = []
        for index in trip_indices:
            destination = self.df.iloc[index]
            # Convert budget to float for comparison
            budget = float(destination['Budget'])
            # Check if the budget is within the user's budget
            if budget <= user_preferences['Budget']:
                recommended_destinations.append({
                    'Destination': destination['Destination'],
                    'Budget': budget
                })
                # Break if we have found enough recommendations
                if len(recommended_destinations) == top_n:
                    break
        return recommended_destinations

# Load the model
model = DestinationRecommendationModel()
checkpoint_filename = '/content/destination_recommendation_checkpoint.pkl'

try:
    model.load_checkpoint(checkpoint_filename)
    print("Model checkpoint loaded successfully!")
except:
    print("Error loading model checkpoint. Please check if the checkpoint file exists.")


# Ensure you have already loaded the necessary modules and defined the DestinationRecommendationModel class

# Create form elements for user preferences
age = widgets.IntSlider(description='Age:', min=0, max=100, value=30)
gender = widgets.Dropdown(options=['Male', 'Female', 'Other'], description='Gender:')
nationality = widgets.Text(description='Nationality:')
accommodation_type = widgets.Dropdown(options=['Hotel', 'Hostel', 'Airbnb', 'Other'], description='Accommodation Type:')
transportation_type = widgets.Dropdown(options=['Flight', 'Train', 'Bus', 'Other'], description='Transportation Type:')
budget = widgets.FloatSlider(description='Budget:', min=0, max=3000, value=1000)

# Create a button to trigger recommendation
button = widgets.Button(description='Get Recommendations')

# Define the function to get recommendations
def get_recommendations(b):
    user_preferences = {
        'Traveler age': age.value,
        'Traveler gender': gender.value,
        'Traveler nationality': nationality.value,
        'Accommodation type': accommodation_type.value,
        'Transportation type': transportation_type.value,
        'Budget': budget.value
    }
    recommendations = model.recommend_destination(user_preferences)
    print("Recommended Destinations based on User Preferences:")
    for destination in recommendations:
        print(destination['Destination'], destination['Budget'])

# Attach the function to the button
button.on_click(get_recommendations)

# Display the form and button
display(age, gender, nationality, accommodation_type, transportation_type, budget, button)



Model checkpoint loaded successfully!


IntSlider(value=30, description='Age:')

Dropdown(description='Gender:', options=('Male', 'Female', 'Other'), value='Male')

Text(value='', description='Nationality:')

Dropdown(description='Accommodation Type:', options=('Hotel', 'Hostel', 'Airbnb', 'Other'), value='Hotel')

Dropdown(description='Transportation Type:', options=('Flight', 'Train', 'Bus', 'Other'), value='Flight')

FloatSlider(value=1000.0, description='Budget:', max=3000.0)

Button(description='Get Recommendations', style=ButtonStyle())

Recommended Destinations based on User Preferences:
Greece 1000.0
London, UK 1000.0
Recommended Destinations based on User Preferences:
Paris 1300.0
Japan 1300.0
Paris 1300.0
Rio de Janeiro, Brazil 950.0
Recommended Destinations based on User Preferences:
Paris 1300.0
Japan 1300.0
Bangkok 900.0
Bangkok 1000.0
